# Homography - Part 2A
Part 2A explores the geometry of a single camera. In 2B the goal is to use a set of correspondance points to estimate a transformation matrix from a plane's 3D space to camera space and use that matrix to project some other points into camera space.

Two components that we need for 2B are built, a method to estimate that transformation and a method that can project points into camera image space.

First, the projection method, `projectiveCamera` to find the image space coordinates, `XImCart`, of a set of 3D world coordinates, `XCart`, given a camera intrinsics matrix `K` and an extrinsics matrix `T`.

The second component is a method to estimate a Eucledian transformation, `TEst`, that translates from a plane's 3D coordinate space to 3D camera space by utilizing a given set of points in camera image space, `XImCart`, and a set of corresponding points in world space, `XCart`.

Estimating the camera pose will involve calculating a homography, functions from part 1A/1B are used.

## Import libraries 

In [1]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

In [2]:
def projectiveCamera(K,T,XCart):
    # The goal of this function is to project points in XCart through projective camera
    # defined by intrinsic matrix K and extrinsic matrix T. In essence, this function takes a set of points 
    # in 3D world space, XCart, and projects them into camera image space by applying the extrinsic matrix T 
    # and then applying the intrinsic matrix K.
    # 
    # There are three steps.
    # 1) Move from world space to camera space. 
    #            camera space points = extrinsics T * world space points 
    #
    # 2) Applying the intrinsics matrix to the camera space points after normalizing
    #           homogeneous image space points = K * normalized camera space points
    # 
    # 3) Move to image space cartesian points from image space homogeneous points, involves a 
    # normalization using the third row.
    
    XImCart = []

    # Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    XHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)

    # Apply extrinsic matrix to XHom, to move to frame of reference of camera
    xCamHom = T @ XHom

    # Project points into normalized camera coordinates xCamHom (remove 4th row)
    xCamHom = np.delete(xCamHom, 3, 0)

    # Move points to image coordinates xImHom by applying intrinsic matrix
    XImHom = K @ xCamHom

    # Convert points back to Cartesian coordinates xImCart
    XImCart = XImHom[0:2,:]/np.tile([XImHom[2,:]],(2,1))
    
    return XImCart


## Camera Projection

The function that transform from 3D world space, `XCart`, to camera image space, `XImCart`, using an extrinsics matrix `T` and an intrinsics matrix `K` that are provided. The previous block houses this function.

The result here is the cartesian image space point coordinates, `XImCart`, of the 3D points `XCart`. If `XCart` represents a box in the world then we now know where the box's vertices would land in image space.

In [3]:
# Assume the camera intrinsics matrix K is known and has values:
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# Assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are five known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])

# Assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])
# T houses a rotation matrix and a translation matrix. The last row is for homogeneous point calculation.


# Use the general pin-hole projective camera model to estimate 
# where the four points on the plane will appear in the image.
XImCart = projectiveCamera(K,T,XCart)

### Functions copied form 1A and 1B

In [4]:
def solveAXEqualsZero(A):
    # Using SVD to solve Ah = 0.
    U,L,V_T = np.linalg.svd(A)  # Compute SVD
    h = V_T.T[:,-1]  # Get the last column of V
    return h

In [5]:
def calcBestHomography(pts1Cart, pts2Cart):
    
    # This function apply the direct linear transform (DLT) algorithm to calculate the best 
    # homography that maps the cartesian points in pts1Cart to their corresonding matching cartesian poitns 
    # in pts2Cart.
    
    # This function calls solveAXEqualsZero.

    n_points = pts1Cart.shape[1]  # 5
    
    H = np.identity(3)

    # First convert points into homogeneous representation.
    pts1Hom = np.concatenate((pts1Cart, np.ones((1,pts1Cart.shape[1]))), axis=0)
    pts2Hom = np.concatenate((pts2Cart, np.ones((1,pts2Cart.shape[1]))), axis=0)
    # Then construct the matrix A, size (n_points * 2, 9)
    A = np.zeros((n_points * 2, 9))
    # Extract u, v, x, and y
    u = pts2Hom[0, :]
    v = pts2Hom[1, :]
    x = pts1Hom[0, :]
    y = pts1Hom[1, :]

    # Construct the matrix A
    for i in range(n_points):
        A[2*i,:]= [0,0,0,-u[i],-v[i],-1,y[i]*u[i],y[i]*v[i],y[i]]       
        A[2*i+1,:]= [u[i],v[i],1,0,0,0,-x[i]*u[i],-x[i]*v[i],-x[i]]

    # Solve Ah = 0 using solveAXEqualsZero and get h.
    h = solveAXEqualsZero(A)

    # Reshape h into the matrix H, values of h go first into rows of H
    H = h.reshape(H.shape)
    return H

In [6]:
def estimatePlanePose(XImCart,XCart,K):
    # The goal of this function is to estimate the pose of a plane relative to camera (extrinsic matrix)
    # given points in image space xImCart, points in 3D world space XCart, and an intrinsics matrix K.
    
    T = []

    # Convert Cartesian image points XImCart to homogeneous representation XImHom
    XImHom = np.concatenate((XImCart, np.ones((1,XImCart.shape[1]))), axis=0)
    
    # Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    XCamHom = np.linalg.inv(K) @ XImHom
    
    # Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    XCamCart = XCamHom[0:2,:]/np.tile([XCamHom[2,:]],(2,1))
    H = calcBestHomography(XCamCart, XCart) 
     
    # Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD.
    R = np.zeros((3, 3)) 
    U,L,V_T = np.linalg.svd(H[:,0:2]) 
    R[:,:2] = U @ np.array([[1,0],[0,1],[0,0]]) @ V_T

    # Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    R[:,2] = np.cross(R[:,0],R[:,1])
        
    # TO DO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    if not np.linalg.det(R) > 0:
        R[:,-1] = -R[:,-1]
    
    # Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    k = np.sum(H[:,:2]/R[:,:2])/6 
    t = H[:,2].reshape((3,1))/k
    
    # Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[2] < 0:
        t *= -1
        R[:,:2] *= -1
            
    # Assemble transformation into matrix form
    T = np.hstack((R,t))
    T = np.vstack((T, np.array([0,0,0,1])))
    
    return T 

## Estimating an Extrinsics Matrix, T

The problem: Given an instrinsics matrix `K`, a set of 3D world points `XCart`, and a set of corresponding image space coordinates in `XImCart`. `K` and `XCart` have already been defined a few cells back and `XImCart` has been calculated by virtue of the exercise completed with camera projection. What is unknown is an extrinsics matrix, `T`, which needs to estimated to fill in `estimatePlanePose` and return `TEst`.

In [7]:
# Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart
noiseLevel = 0.1
XImCartNoisy = XImCart + np.random.normal(0, noiseLevel, XImCart.shape) #add noise here

# Estimate the extrinsic matrix 
TEst = estimatePlanePose(XImCartNoisy,XCart,K)

# TEst should closely resemble the groundtruth, T.
np.set_printoptions(precision=3)
print(TEst)
print("\n")
print(T)


[[ 9.855e-01 -4.985e-02  1.624e-01  4.613e+01]
 [-1.627e-01 -5.519e-01  8.179e-01  7.014e+01]
 [ 4.884e-02 -8.324e-01 -5.520e-01  5.021e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


[[ 9.851e-01 -4.920e-02  1.619e-01  4.600e+01]
 [-1.623e-01 -5.520e-01  8.181e-01  7.000e+01]
 [ 4.900e-02 -8.324e-01 -5.518e-01  5.009e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


It can be seen that the estimated extrinsics matrix TEst closely resemble the groundtruth T as expected. Small error results from the noise added.